In [1]:
import paramiko
import mysql.connector	
from mysql.connector import connect, Error
import sshtunnel
import time

# SSH Configuration
ssh_host = '35.212.230.135'
ssh_port = 22
ssh_user = 'dev'
ssh_password = 'Inf2003dev'

# MySQL Configuration
mysql_host = 'localhost'  # or the IP address of your MySQL server
mysql_port = 3306
mysql_user = 'sqluser'
mysql_password = 'Inf2003user'
mysql_database = 'olist_db'

In [ ]:
# Establish SSH connection
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())


ssh_client.connect(ssh_host, ssh_port, ssh_user, ssh_password)
print("SSH Connection Established")

# Establish MySQL connection through SSH tunnel
tunnel = sshtunnel.SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_password=ssh_password,
        remote_bind_address=(mysql_host, mysql_port),
)
tunnel.start()
print("SSH Tunnel Established")

connection = mysql.connector.connect(
        user=mysql_user,
        host=mysql_host,
        port=tunnel.local_bind_port,
        password=mysql_password,
        database=mysql_database,
        use_pure=True,  
                
)
print("MySQL Connection Established")


In [1]:
# Your MySQL query here
#cursor = connection.cursor()
query = "SELECT p.product_category_name, COUNT(DISTINCT oi.order_id) AS Num_orders, SUM(oi.price) AS Revenue \
FROM olist_order_items_dataset oi \
JOIN olist_products_dataset p ON oi.product_id = p.product_id \
JOIN olist_orders_dataset o ON oi.order_id = o.order_id \
WHERE o.order_status <> 'canceled' AND o.order_delivered_customer_date IS NOT NULL \
GROUP BY p.product_category_name \
ORDER BY Revenue DESC; \
"


In [ ]:
timelist = []
num = 0
while num < 10:
    seconds = time.time()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    timetaken = time.time() - seconds

    # for r in reuslt:
    #     print(r)

    timelist.append(timetaken)
    num += 1
    print(num)

for i in timelist:
    print(i)

In [ ]:
cursor = connection.cursor()
seconds = time.time()
cursor.execute(query)
timetaken = time.time() - seconds
result = cursor.fetchall()
cursor.close()
print(timetaken)

In [8]:
connection.close()
# Close SSH connection
ssh_client.close()